In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
from pylab import *
import matplotlib
#import plplot
from scipy import stats
#from io import StringIO
print(pd.__version__)
#AttributeError: 'Series' object has no attribute 'to_numpy'
import re

import math
import astropy as astro
import scipy.ndimage as spimage
from astropy.io import fits, ascii
from astropy.table import Table, QTable
from astropy.table import join as Table_join
from astropy.table import unique as tab_unique
from astropy.cosmology import WMAP9 as cosmo
import matplotlib as mpl
#from numpy import std as biweight_midvariance
import matplotlib.cm as cm

from scipy import optimize
from scipy.stats import gaussian_kde
from matplotlib import colors
import scipy.ndimage as ndimage
from collections import Counter
from matplotlib.legend import Legend
import matplotlib.patches as patches
from matplotlib import pyplot
#
#
#
import extinction


#
# seaborn
#import pandas.util.testing as tm
import seaborn as sns

#
# Jorge
#

from my_tools.sel2dhist_xy import sel2dhist_xy


#
# 3D plots!
#
import numpy as np
import scipy.linalg
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

#biweight_midvariance


import warnings
warnings.simplefilter("ignore")

#
# my_utils
#
from my_tools.my_utils import *

#
#
#from ndtest import *
from sklearn.decomposition import PCA

#
#
#
from scipy.optimize import leastsq, fmin
import matplotlib.animation as animation

#
# Carlos Color map
#
from my_tools.cmaps_CLC import vel_map

from astropy.table import setdiff

from scipy.stats import ks_2samp
#import ndtest

import sys

from matplotlib.colors import LinearSegmentedColormap


1.1.1
1.1.1


In [64]:

from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
from astropy import units as u

In [86]:
def query_hyperleda(ra, dec):
    """
    Query HyperLeda for a galaxy's position angle and ellipticity (b/a ratio) based on coordinates.
    
    Parameters:
        ra (float): Right Ascension in degrees.
        dec (float): Declination in degrees.
        
    Returns:
        dict: A dictionary with position angle and ellipticity (b/a ratio).
    """
    # HyperLeda catalog via Vizier
    catalog = "VII/237"

    # Create SkyCoord object for the input coordinates
    coords = SkyCoord(ra=ra * u.deg, dec=dec * u.deg, frame='icrs')

    # Convert to HMS/DMS format
    ra_hms = coords.ra.to_string(unit=u.hour, sep=':', precision=2)  # RA in HMS
    dec_dms = coords.dec.to_string(unit=u.deg, sep=':', alwayssign=True, precision=2)  # Dec in DMS

    coords_trans = SkyCoord(ra_hms, dec_dms, unit=(u.hourangle, u.deg))
#    print(ra,dec)

    print(coords_trans)
    # Query Vizier around the given coordinates
    Vizier.ROW_LIMIT = 10  # Limit the number of rows returned
    result = Vizier.query_region(coords_trans, radius=0.1 * u.deg, catalog=catalog)

    if not result:
        return {"Error": "No matching galaxy found in HyperLeda."}
    
    # Extract relevant data
    table = result[0]
    position_angle = table["PA"][0] if "PA" in table.colnames else None
    b_a_ratio = table["e"][0] if "e" in table.colnames else None  # HyperLeda ellipticity
    
    if position_angle is None or b_a_ratio is None:
        return {"Error": "Position angle or b/a ratio not available in HyperLeda for this object."}
    
    # Convert ellipticity to b/a ratio (b/a = 1 - ellipticity)
    b_a_ratio = 1 - b_a_ratio

    return {
        "Position Angle (PA)": position_angle,
        "Ellipticity (b/a)": b_a_ratio
    }

# Example usage
#ra = 10.684  # Right Ascension in degrees
#dec = 41.269  # Declination in degrees

ra=232.652484
dec=42.717198
print(ra,dec)

result = query_hyperleda(ra, dec)
print(result)

232.652484 42.717198
<SkyCoord (ICRS): (ra, dec) in deg
    (232.6525, 42.71719722)>
{'Error': 'Position angle or b/a ratio not available in HyperLeda for this object.'}


In [82]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u

def query_vizier_by_coordinates(ra_hms, dec_dms, catalog="VII/237", radius=0.1):
    """
    Query the VizieR catalog using coordinates in RA (hms) and Dec (dms) format.

    :param ra_hms: RA in 'hh:mm:ss.ssss' format
    :param dec_dms: Dec in '+dd:mm:ss.ssss' format
    :param catalog: VizieR catalog identifier (default: 'VII/118' for HyperLeda)
    :param radius: Search radius in degrees (default: 0.1)
    :return: Table of results or an error message
    """
    # Convert RA/Dec to SkyCoord object
    coords = SkyCoord(ra=ra_hms, dec=dec_dms, unit=(u.hourangle, u.deg))
    
    # Initialize VizieR
    vizier = Vizier(columns=["*", "+_r"])  # Request all columns plus the distance column
    vizier.ROW_LIMIT = -1  # Retrieve all rows

    try:
        # Perform the query
        result = vizier.query_region(coords, radius=radius * u.deg, catalog=catalog)
        
        if len(result) == 0:
            return "No matching objects found in the catalog."
        
        return result[catalog]

    except Exception as e:
        return f"Query failed: {e}"

# Example usage
ra_hms = "15h30m36.5962s"
dec_dms = "+42d43m01.911s"
catalog = "VII/118"  # HyperLeda catalog

result = query_vizier_by_coordinates(ra_hms, dec_dms, catalog)

# Print the results
if isinstance(result, str):  # If error message
    print(result)
else:
    print(result)

Query failed: TableLists can only be indexed with the named keys and integers.


In [9]:
def plot_diagnostic_diagram_single(type_plot: str, statistic: str,
                                   prop: str,
                                   data: pd.DataFrame,
                                   ax: plt.Axes,
                                   vmin=None, vmax=None,
                                   histogram=False) -> mpl.image.AxesImage:
    """
    Diagnostic diagrams plotting auxiliary function
    """
    ylabel = 'log_OIII_Hb_cen'
    if type_plot == 'O3N2':
        xlabel = 'log_NII_Ha_cen'
    elif type_plot == 'O3S2':
        xlabel = 'log_SII_Ha_cen'
    elif type_plot == 'O3O1':
        xlabel = 'log_OI_Ha_cen'
    elif type_plot == 'O3O2':
        xlabel = 'log_OII_Hb_cen'
    x_data = data[xlabel]
    y_data = data[ylabel]
    z_data = data[prop]
    mean_prop = np.mean(z_data)
    std_prop = np.std(z_data)
    # print(prop, mean_prop, std_prop)
    if statistic == 'mean':
        if vmin is None:
            if prop == 'Av':
                vmin = 0
            else:
                vmin = mean_prop - 3*std_prop
        if vmax is None:
            if prop == 'Av':
                vmax = 2*mean_prop
            else:
                vmax = mean_prop + 3*std_prop
    elif statistic == 'std':
        if vmin is None:
            vmin = 0
        if vmax is None:
            vmax = 2*std_prop
    plot = diagnostic_diagram_plot(x_data, y_data, z_data, type_plot=type_plot,
                                   vmin=vmin, vmax=vmax, ax=ax,
                                   statistic=statistic)
    ax.xaxis.set_major_locator(MultipleLocator(1))
    ax.yaxis.set_major_locator(MultipleLocator(1))
    ax.xaxis.set_minor_locator(MultipleLocator(0.1))
    ax.yaxis.set_minor_locator(MultipleLocator(0.1))
    ax.tick_params(axis='both', which='both', direction='in', top=True,
                   right=True)
    if histogram:
        axis_ins = ax.inset_axes([0.05, 0.10, 0.35, 0.15])
        counts, bins, _ = axis_ins.hist(z_data, bins=60, density=True,
                                        range=(vmin, vmax), color='k')
        axis_ins.plot([mean_prop, mean_prop], [0, max(counts)], 'r--', lw=1)
        msg = r"${}\pm{}$".format(np.round(mean_prop, 2),
                                  np.round(std_prop, 2))
        if prop == 'f_y':
            y_pos = mean_prop+.05
        else:
            y_pos = mean_prop-1.5*np.abs(mean_prop)
#            y_pos = mean_prop+.15
        axis_ins.text(y_pos, max(counts)*(1.1), msg, {'fontsize': 14})
        axis_ins.spines['top'].set_visible(False)
        axis_ins.spines['right'].set_visible(False)
        axis_ins.spines['left'].set_visible(False)
        axis_ins.set_yticks([])
        #axis_ins.set_xticklabels([])
        axis_ins.patch.set_alpha(0.1)
        #axis_ins.set_xticks(get_ticks_hist(prop))
    return plot


In [10]:
#
def my_scatter_single(ax,x_par,y_par,c_par,x_min,x_max,y_min,y_max,c_min,c_max,x_label,y_label,xf_min=0.0,yf_min=0.0,den_par_min=0.85,bin_size=0.3,bin_number=2,c_color="black",c_color2="white",error=0.05,MC=10,cm='Wistia_r', cm_r='Wistia', title='', fit_title='', if_contour=1, if_scatter=1, if_bin=1, if_plot=0, max_c=0.95, Delta=0.15, if_axis=0):
#def my_scatter_single(ax,x_par,y_par,c_par,x_min,x_max,y_min,y_max,c_min,c_max,x_label,y_label,xf_min=0.0,yf_min=0.0,den_par_min=0.95,bin_size=0.3,bin_number=2,c_color="black",c_color2="white",error=0.05,MC=10,cm='YlOrRd_r', cm_r='YlOrRd', title='', fit_title='', if_contour=1, if_scatter=1, if_bin=1):
#def my_scatter_single(ax,x_par,y_par,c_par,x_min,x_max,y_min,y_max,c_min,c_max,x_label,y_label,xf_min=0.0,yf_min=0.0,den_par_min=0.95,bin_size=0.3,bin_number=2,c_color="black",c_color2="white",error=0.05,MC=10,cm='YlOrRd_r', cm_r='YlOrRd', title='', fit_title=''):
#    cm = califa_vel_r
#    cm='Wistia_r'
    if (xf_min==0.0):
        xf_min=x_min
    if (yf_min==0.0):
        yf_min=y_min
#    MC=5
#    error=0.05
    N_min=2
#    Delta=0.15
#    den_par_min=0.9
    nbins=60
    lEW_cut=0.78 #0.78
    n_obj_org=len(x_par)
    mask = x_par>-20
    n_obj=len(x_par[mask])
    mask_SFGs = mask & (c_par>lEW_cut)
    #
    # Density near a point
    #
    x_plt, y_plt = x_par[mask_SFGs], y_par[mask_SFGs]
    n_sf=len(x_par[mask_SFGs])
    #    nbins=40
    counts, xbins, ybins = np.histogram2d(x_plt, y_plt, bins=nbins,
        normed=True,
        range=[[np.nanmin(x_plt),np.nanmax(x_plt)],[np.nanmin(y_plt),np.nanmax(y_plt)]])
                                            #    counts /= counts.max()
#    print(xbins,ybins)
    counts=ndimage.gaussian_filter(counts, sigma=1, order=0)
    counts /= counts.max()
    sum_total=counts.sum()
    vals_new=[]
    levels_new=[]
    for idx,cuts in enumerate(np.arange(0.00,1.0,0.01)):
        mask_now= counts>cuts
        levels_new.append(cuts)
        vals_new.append(counts[mask_now].sum()/sum_total)
        #print(idx,levels[idx],vals[idx])
    vals_cont=np.array([max_c,0.80,0.40,-0.01])
    levels_cont=np.interp(vals_cont,np.array(levels_new),np.array(vals_new))
    
#    figure=ax.scatter(x_par, y_par, c=c_par, vmin=c_min,vmax=c_max,alpha=0.4,edgecolor='none',cmap=cm)
    counts_rot=np.rot90(counts,3)
    xbins=xbins+0.5*(x_max-x_min)/nbins
    ybins=ybins+0.5*(y_max-y_min)/nbins
    flip_counts_rot=np.fliplr(counts_rot)
    
  
    den_par=np.zeros(len(x_par))
    for i in range(len(x_par)):
        den_par[i]=1.0
        if ((np.isfinite(x_par[i])) and (np.isfinite(y_par[i]))):
            i_x=np.argmin(np.abs(xbins-x_par[i]))
            i_y=np.argmin(np.abs(ybins-y_par[i]))
            if ((i_x>0) and (i_x<nbins) and (i_y>0) and (i_y<nbins)):
                den_par[i]=np.interp(counts[i_x,i_y],np.array(levels_new),np.array(vals_new))



    if (if_scatter==1):
        figure=ax.scatter(x_par, y_par, s=10, c=den_par,\
                          vmin=0,vmax=1,alpha=0.25,edgecolor='lightyellow',cmap=cm,\
                          rasterized=True,label=title)
        figure=ax.scatter(x_par, y_par, s=12, c=den_par,\
                          vmin=0,vmax=1,alpha=0.25,edgecolor='none',cmap=cm,\
                          rasterized=True)

    if (if_contour==1):
        p_cont2=ax.contourf(xbins[0:nbins],ybins[0:nbins],flip_counts_rot,\
                             levels_cont,cmap=cm_r,alpha=0.8,linewidths=2)
#        p_cont2=ax.contour(xbins[0:nbins],ybins[0:nbins],flip_counts_rot,\
#                           levels_cont,cmap=cm_r,alpha=0.8,linewidths=2)



#
# We mask low density points
#
#    print("# den_par_min=",den_par_min)
    mask_SFGs = mask_SFGs & (den_par<den_par_min) & \
    np.logical_not(np.isnan(x_par)) & np.isfinite(x_par) & \
    np.logical_not(np.isnan(y_par)) & np.isfinite(y_par)  

    x_sf=x_par[mask_SFGs]
    y_sf=y_par[mask_SFGs]
    
#    figure=ax.scatter(x_par, y_par, c=den_par, vmin=0,vmax=1,alpha=0.4,edgecolor='none',cmap=cm)
    
    bin1 , min1 , max1= Delta,x_min,x_max
    m1, sM1, sD1 = binning_M(x_sf, y_sf, bin1 , min1 , max1, N_min, bin_size, bin_number )
    if (if_bin==1):
        p1 = ax.errorbar(m1, sM1, yerr= sD1, markersize = 10,\
                         markerfacecolor =c_color2, markeredgecolor = 'black', \
                         fmt = 'o', ecolor='black', elinewidth = 1, label = '', zorder=2, alpha=1.0)


#    x_sf=x_par[mask_SFGs]
#    y_sf=y_par[mask_SFGs]
    bin1 , min1 , max1= Delta,xf_min,x_max
    funct    = lambda x,a,b: a + b * x
    START=[-1,1.0]
    pa1=np.zeros(2)
    ea1=np.zeros(2)
#    print('#vals to fit =',m1,sM1,sD1)
    if m1.size > 2:
        m1, sM1, sD1 = binning_M(x_sf, y_sf, bin1 , min1 , max1, N_min, 0.3,2 )
        pa1, ea1  = optimize.curve_fit(funct, m1, sM1, sigma=np.sqrt(sD1), p0 = START )
        a_pa=np.zeros((MC,2))
        a_ea=np.zeros((MC,2,2))
        a_rc=np.zeros((MC,2,2))
        for iMC in range(MC):
            e_x_sf=np.abs(2*error-error*(x_sf-x_min)/(x_max-x_min))
            e_y_sf=np.abs(2*error-error*(y_sf-y_min)/(y_max-y_min))
            
#            print('e_y=',e_y_sf,len(e_y_sf))
#            print('e-x=',e_x_sf,len(e_x_sf))
            x_sf_now=x_sf+np.random.normal(loc=0.0,scale=e_x_sf,size=len(e_x_sf))
            y_sf_now=y_sf+np.random.normal(loc=0.0,scale=e_y_sf,size=len(e_y_sf))
            mNOW, sMNOW, sDNOW = binning_M(x_sf_now, y_sf_now, bin1 , min1 , max1, N_min, 0.3,2 )
            rcNOW=np.corrcoef(x_sf_now, y_sf_now)            
            paNOW, eaNOW  = optimize.curve_fit(funct, mNOW, sMNOW, sigma=np.sqrt(sDNOW), p0 = START )
#            print('Coeffs lin fit: ',round(paNOW[0],3),round(np.sqrt(np.diag(eaNOW))[0],3),round(paNOW[1],3),round(np.sqrt(np.diag(eaNOW))[1],3),round(rcNOW[0,1],3))    
            a_pa[iMC,:]=paNOW
            a_ea[iMC,:,:]=eaNOW
            a_rc[iMC,:,:]=rcNOW
#        print(pa1,ea1)
#        test=np.mean(a_pa,axis=(0,1))
#        print(test)
        np.mean(a_pa, axis=(0), out=pa1)
        np.mean(a_ea, axis=(0), out=ea1)
        e_pa1=np.std(a_pa, axis=(0))
        rc=np.mean(a_rc, axis=(0))
        ea1[0,0]=ea1[0,0]+e_pa1[0]
        ea1[1,1]=ea1[1,1]+e_pa1[1]
        
        
        #        pa1=a_pa.mean(axis=(1,2))
#        ea1=a_ea.mean(axis=(1,2))
#        ea1=np.mean(a_ea,axis=2)
        
#        pa1, ea1  = optimize.curve_fit(funct, m1, sM1, sigma=sD1, p0 = START )
#round(pa1[0],3),round(np.sqrt(np.diag(ea1))[0],3),round(pa1[1],3),round(np.sqrt(np.diag(ea1))[1],3),round(rc[0,1],3)

        if (xf_min!=x_min):
            yl_plot=np.linspace(y_min,y_max,10)
            xl_plot=xf_min+0.0*yl_plot
            rect=patches.Rectangle((x_min, y_min), xf_min-x_min, y_max-y_min, facecolor="black", alpha=0.05,zorder=3)
            ax.add_patch(rect)
#        x_plot=np.linspace(np.min(m1)-1.5*bin1, np.max(m1)+1.5*bin1, 10)
        x_plot=np.linspace(xf_min-0.15*(x_max-x_min), x_max-0.15*(x_max-x_min), 10)
        y_plot=funct(x_plot,pa1[0],pa1[1])
        if (if_plot==1):
            p_fit, =ax.plot(x_plot, y_plot, '-', linewidth = 3, markerfacecolor =c_color  ,\
                            color =c_color, zorder=3, alpha=0.8)

        
#        p_fit=ax.plot(xl_plot, yl_plot, '-', linewidth = 2, markerfacecolor ='grey'  , color ='grey', zorder=2)        
#        print("PASO")
    mask_x_sf=x_sf>xf_min
    x_sf=x_sf[mask_x_sf]
    y_sf=y_sf[mask_x_sf]
    rc=np.corrcoef(x_sf, y_sf)
    delta_y_par=y_sf-(pa1[0]+pa1[1]*x_sf)
    s_y_par = biweight_midvariance(y_sf[np.isfinite(y_sf)])
    s_dy_par = biweight_midvariance(delta_y_par[np.isfinite(delta_y_par)])

    print('Mean Coeff.: ',round(pa1[0],3),round(np.sqrt(np.diag(ea1))[0],3),round(pa1[1],3),round(np.sqrt(np.diag(ea1))[1],3),round(rc[0,1],3),round(s_y_par,3),round(s_dy_par,3),n_obj,n_sf)
    #    print('Stddev: ',round(s_y_par,3),round(s_dy_par,3))
    #     if (if_contour==1):
    if ((len(title)>0) and (if_contour==1)):
        p_cont2.collections[0].set_label(title)
    if ((len(fit_title)>0) and (if_plot==1)):
        p_fit.set_label(fit_title)

#    p_cont2.collections[0].set_label(title)        
    if (if_axis==1):
        ax.set_xlim([x_min,x_max])
        ax.set_ylim([y_min,y_max])    
        ax.set_xlabel(x_label)
        ax.set_ylabel(y_label)
        ax.minorticks_on()
        ax.xaxis.set_ticks_position('both')
        ax.yaxis.set_ticks_position('both')
        ax.tick_params(which='both',direction="in")
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')

    return pa1,ea1,s_y_par,s_dy_par,rc





In [11]:
import matplotlib.colors as mcolors
color_cm_now = vel_map()
color_cm_r = color_cm_now.reversed()



In [12]:
from matplotlib import rcParams as rc
rc.update({'font.size': 20,\
           'font.weight': 900,\
           'text.usetex': True,\
           'path.simplify'           :   True,\
           'xtick.labelsize' : 20,\
           'ytick.labelsize' : 20,\
#           'xtick.major.size' : 3.5,\
#           'ytick.major.size' : 3.5,\
           'axes.linewidth'  : 2.0,\
               # Increase the tick-mark lengths (defaults are 4 and 2)
           'xtick.major.size'        :   6,\
           'ytick.major.size'        :   6,\
           'xtick.minor.size'        :   3,\
           'ytick.minor.size'        :   3,\
           'xtick.major.width'       :   1,\
           'ytick.major.width'       :   1,\
           'lines.markeredgewidth'   :   1,\
           'legend.numpoints'        :   1,\
           'xtick.minor.width'       :   1,\
           'ytick.minor.width'       :   1,\
           'legend.frameon'          :   False,\
           'legend.handletextpad'    :   0.3,\
           'font.family'    :   'serif',\
           'mathtext.fontset'        :   'stix',\
           'axes.facecolor' : "w",\
           
          })


In [13]:
hdu_MaNGA=fits.open('tables/SDSS17Pipe3D_v3_1_1.fits')
tab_MaNGA=Table(hdu_MaNGA[1].data)     
list_columns(tab_MaNGA.colnames)

name                                a_ZH_T90                            flux_[NII]6548.05_Re_fit            U_Dors_O32_alpha_fit                
plate                               T95                                 e_flux_[NII]6548.05_Re_fit          e_U_Dors_O32_alpha_fit              
ifudsgn                             ZH_T95                              flux_[NII]6548.05_alpha_fit         U_Dors_S_Re_fit                     
plateifu                            ZH_Re_T95                           e_flux_[NII]6548.05_alpha_fit       e_U_Dors_S_Re_fit                   
mangaid                             a_ZH_T95                            flux_Halpha6562.85_Re_fit           U_Dors_S_alpha_fit                  
objra                               T99                                 e_flux_Halpha6562.85_Re_fit         e_U_Dors_S_alpha_fit                
objdec                              ZH_T99                              flux_Halpha6562.85_alpha_fit        U_Mor16_O23_fs_Re_fit 

In [14]:
tab_MaNGA['ID']=tab_MaNGA['mangaid']

In [32]:
#ascii.read.help() 

AttributeError: 'function' object has no attribute 'help'

In [43]:
tab_cubes=ascii.read('tables/list_cubes.txt')
print(len(tab_cubes))
IDXs=[]
for name_now in tab_cubes['cubename']:
    IDX = name_now.replace('/mnt/naschiripa/KILOGAS/cubes/','')
    IDX = IDX.replace('.cube.fits.gz','')
    IDXs.append(IDX)
IDXs=np.array(IDXs)
print(len(IDXs))
print(IDXs)

451
451
['KG-MaNGA-1-1037' 'KG-MaNGA-1-109073' 'KG-MaNGA-1-109112'
 'KG-MaNGA-1-109167' 'KG-MaNGA-1-109234' 'KG-MaNGA-1-1093'
 'KG-MaNGA-1-109679' 'KG-MaNGA-1-1151' 'KG-MaNGA-1-116700'
 'KG-MaNGA-1-116972' 'KG-MaNGA-1-116989' 'KG-MaNGA-1-117014'
 'KG-MaNGA-1-117026' 'KG-MaNGA-1-117104' 'KG-MaNGA-1-117122'
 'KG-MaNGA-1-117381' 'KG-MaNGA-1-117388' 'KG-MaNGA-1-117465'
 'KG-MaNGA-1-117663' 'KG-MaNGA-1-117936' 'KG-MaNGA-1-117938'
 'KG-MaNGA-1-117946' 'KG-MaNGA-1-117981' 'KG-MaNGA-1-118034'
 'KG-MaNGA-1-118083' 'KG-MaNGA-1-118178' 'KG-MaNGA-1-118217'
 'KG-MaNGA-1-118223' 'KG-MaNGA-1-118238' 'KG-MaNGA-1-118337'
 'KG-MaNGA-1-118369' 'KG-MaNGA-1-118444' 'KG-MaNGA-1-118480'
 'KG-MaNGA-1-1243' 'KG-MaNGA-1-1258' 'KG-MaNGA-1-1278' 'KG-MaNGA-1-138167'
 'KG-MaNGA-1-138298' 'KG-MaNGA-1-138401' 'KG-MaNGA-1-139929'
 'KG-MaNGA-1-14151' 'KG-MaNGA-1-14162' 'KG-MaNGA-1-144455'
 'KG-MaNGA-1-144478' 'KG-MaNGA-1-144884' 'KG-MaNGA-1-144889'
 'KG-MaNGA-1-178794' 'KG-MaNGA-1-178894' 'KG-MaNGA-1-179071'
 'KG-MaNGA

In [50]:
hdu=fits.open('tables/KILOGAS_global_catalog.fits')
tab=Table(hdu[1].data)    
IDX_tab=[]
for tab_now in tab:
    IDX = 'KG-'+tab_now['SURVEY']+'-'+tab_now['ID']
    IDX = IDX.replace('MANGA','MaNGA')
    IDX_tab.append(IDX)
IDX_tab=np.array(IDX_tab)
#print(IDX_tab)


#set1, set2 = set(IDXs), set(IDX_tab)
#diff = set1-set2
#print(list(diff))



In [72]:
N=2
ra = tab['RA'][N]
dec = tab['DEC'][N]

ra=232.652484
dec=42.717198
print(ra,dec)


query_hyperleda(ra, dec)

232.652484 42.717198


{'Error': 'Position angle or b/a ratio not available in HyperLeda for this object.'}

In [62]:
mask_IDX = []
for IDX in IDX_tab:
    mask_now = IDX in IDXs
#    print(IDX,IDXs)
#    #any(IDX in item for item in IDXs)
#    print(mask_now)
    mask_IDX.append(mask_now)
mask_IDX = np.array(mask_IDX)
print(IDX_tab[~mask_IDX])
print(tab['ID','SURVEY'][~mask_IDX])
ID_miss = tab['ID'][~mask_IDX]
print(ID_miss)
tab_MaNGA[tab_MaNGA['ID']=='ID_miss']

['KG-MaNGA-1-98506']
   ID   SURVEY
------- ------
1-98506  MANGA
   ID  
-------
1-98506


name,plate,ifudsgn,plateifu,mangaid,objra,objdec,log_SFR_Ha,FoV,Re_kpc,e_log_Mass,e_log_SFR_Ha,log_Mass,log_SFR_ssp,log_NII_Ha_cen,e_log_NII_Ha_cen,log_OIII_Hb_cen,e_log_OIII_Hb_cen,log_SII_Ha_cen,e_log_SII_Ha_cen,log_OII_Hb_cen,e_log_OII_Hb_cen,EW_Ha_cen,e_EW_Ha_cen,ZH_LW_Re_fit,e_ZH_LW_Re_fit,alpha_ZH_LW_Re_fit,e_alpha_ZH_LW_Re_fit,ZH_MW_Re_fit,e_ZH_MW_Re_fit,alpha_ZH_MW_Re_fit,e_alpha_ZH_MW_Re_fit,Age_LW_Re_fit,e_Age_LW_Re_fit,alpha_Age_LW_Re_fit,e_alpha_Age_LW_Re_fit,Age_MW_Re_fit,e_Age_MW_Re_fit,alpha_Age_MW_Re_fit,e_alpha_Age_MW_Re_fit,Re_arc,DL,DA,PA,ellip,log_Mass_gas,vel_sigma_Re,e_vel_sigma_Re,log_SFR_SF,log_SFR_D_C,OH_O3N2_cen,e_OH_O3N2_cen,OH_N2_cen,e_OH_N2_cen,OH_ONS_cen,e_OH_ONS_cen,OH_R23_cen,e_OH_R23_cen,OH_pyqz_cen,e_OH_pyqz_cen,OH_t2_cen,e_OH_t2_cen,OH_M08_cen,e_OH_M08_cen,OH_T04_cen,e_OH_T04_cen,OH_dop_cen,e_OH_dop_cen,OH_O3N2_EPM09_cen,e_OH_O3N2_EPM09_cen,log_OI_Ha_cen,e_log_OI_Ha_cen,Ha_Hb_cen,e_Ha_Hb_cen,log_NII_Ha_Re,e_log_NII_Ha_Re,log_OIII_Hb_Re,e_log_OIII_Hb_Re,log_SII_Ha_Re,e_log_SII_Ha_Re,log_OII_Hb_Re,e_log_OII_Hb_Re,log_OI_Ha_Re,e_log_OI_Ha_Re,EW_Ha_Re,e_EW_Ha_Re,Ha_Hb_Re,e_Ha_Hb_Re,log_NII_Ha_ALL,e_log_NII_Ha_ALL,log_OIII_Hb_ALL,e_log_OIII_Hb_ALL,log_SII_Ha_ALL,e_log_SII_Ha_ALL,log_OII_Hb_ALL,e_log_OII_Hb_ALL,log_OI_Ha_ALL,e_log_OI_Ha_ALL,EW_Ha_ALL,e_EW_Ha_ALL,Ha_Hb_ALL,Sigma_Mass_cen,e_Sigma_Mass_cen,Sigma_Mass_Re,e_Sigma_Mass_Re,Sigma_Mass_ALL,e_Sigma_Mass_ALL,T30,ZH_T30,ZH_Re_T30,a_ZH_T30,T40,ZH_T40,ZH_Re_T40,a_ZH_T40,T50,ZH_T50,ZH_Re_T50,a_ZH_T50,T60,ZH_T60,ZH_Re_T60,a_ZH_T60,T70,ZH_T70,ZH_Re_T70,a_ZH_T70,T80,ZH_T80,ZH_Re_T80,a_ZH_T80,T90,ZH_T90,ZH_Re_T90,a_ZH_T90,T95,ZH_T95,ZH_Re_T95,a_ZH_T95,T99,ZH_T99,ZH_Re_T99,a_ZH_T99,log_Mass_gas_Av_gas_OH,log_Mass_gas_Av_ssp_OH,vel_ssp_2,e_vel_ssp_2,vel_Ha_2,e_vel_Ha_2,vel_ssp_1,e_vel_ssp_1,vel_Ha_1,e_vel_Ha_1,log_SFR_ssp_100Myr,log_SFR_ssp_10Myr,vel_disp_Ha_cen,vel_disp_ssp_cen,vel_disp_Ha_1Re,vel_disp_ssp_1Re,log_Mass_in_Re,ML_int,ML_avg,F_Ha_cen,e_F_Ha_cen,R50_kpc_V,e_R50_kpc_V,R50_kpc_Mass,e_R50_kpc_Mass,log_Mass_corr_in_R50_V,e_log_Mass_corr_in_R50_V,log_Mass_gas_Av_gas_log_log,Av_gas_Re,e_Av_gas_Re,Av_ssp_Re,e_Av_ssp_Re,Lambda_Re,e_Lambda_Re,nsa_redshift,nsa_mstar,nsa_inclination,flux_[OII]3726.03_Re_fit,e_flux_[OII]3726.03_Re_fit,flux_[OII]3726.03_alpha_fit,e_flux_[OII]3726.03_alpha_fit,flux_[OII]3728.82_Re_fit,e_flux_[OII]3728.82_Re_fit,flux_[OII]3728.82_alpha_fit,e_flux_[OII]3728.82_alpha_fit,flux_HI3734.37_Re_fit,e_flux_HI3734.37_Re_fit,flux_HI3734.37_alpha_fit,e_flux_HI3734.37_alpha_fit,flux_HI3797.9_Re_fit,e_flux_HI3797.9_Re_fit,flux_HI3797.9_alpha_fit,e_flux_HI3797.9_alpha_fit,flux_HeI3888.65_Re_fit,e_flux_HeI3888.65_Re_fit,flux_HeI3888.65_alpha_fit,e_flux_HeI3888.65_alpha_fit,flux_HI3889.05_Re_fit,e_flux_HI3889.05_Re_fit,flux_HI3889.05_alpha_fit,e_flux_HI3889.05_alpha_fit,flux_HeI3964.73_Re_fit,e_flux_HeI3964.73_Re_fit,flux_HeI3964.73_alpha_fit,e_flux_HeI3964.73_alpha_fit,flux_[NeIII]3967.46_Re_fit,e_flux_[NeIII]3967.46_Re_fit,flux_[NeIII]3967.46_alpha_fit,e_flux_[NeIII]3967.46_alpha_fit,flux_CaII3968.47_Re_fit,e_flux_CaII3968.47_Re_fit,flux_CaII3968.47_alpha_fit,e_flux_CaII3968.47_alpha_fit,flux_Hepsilon3970.07_Re_fit,e_flux_Hepsilon3970.07_Re_fit,flux_Hepsilon3970.07_alpha_fit,e_flux_Hepsilon3970.07_alpha_fit,flux_Hdelta4101.77_Re_fit,e_flux_Hdelta4101.77_Re_fit,flux_Hdelta4101.77_alpha_fit,e_flux_Hdelta4101.77_alpha_fit,flux_Hgamma4340.49_Re_fit,e_flux_Hgamma4340.49_Re_fit,flux_Hgamma4340.49_alpha_fit,e_flux_Hgamma4340.49_alpha_fit,flux_Hbeta4861.36_Re_fit,e_flux_Hbeta4861.36_Re_fit,flux_Hbeta4861.36_alpha_fit,e_flux_Hbeta4861.36_alpha_fit,flux_[OIII]4958.91_Re_fit,e_flux_[OIII]4958.91_Re_fit,flux_[OIII]4958.91_alpha_fit,e_flux_[OIII]4958.91_alpha_fit,flux_[OIII]5006.84_Re_fit,e_flux_[OIII]5006.84_Re_fit,flux_[OIII]5006.84_alpha_fit,e_flux_[OIII]5006.84_alpha_fit,flux_HeI5015.68_Re_fit,e_flux_HeI5015.68_Re_fit,flux_HeI5015.68_alpha_fit,e_flux_HeI5015.68_alpha_fit,flux_[NI]5197.9_Re_fit,e_flux_[NI]5197.9_Re_fit,flux_[NI]5197.9_alpha_fit

In [47]:
def is_value_included_partial(array, value):
    return any(value in item for item in array)

# Example usage
array = ["apple pie", "banana split", "cherry tart"]
value = "banana"

result = is_value_included_partial(array, value)
print(f"Is {value} a substring in any array element? {result}")

Is banana a substring in any array element? <generator object is_value_included_partial.<locals>.<genexpr> at 0x7f5e78b3b120>


In [16]:
tab

KGAS_ID,ID,SURVEY,RA,DEC,Z,LOGMSTAR_GSWLC,LOGMSTAR_ERR_GSWLC,LOGSFR_GSWLC,LOGSFR_ERR_GSWLC,LOGMSTAR_SDSS,LOGSFR_SDSS,LOGMSTAR_BEST,LOGSFR_BEST,N2HA,O3HB,BALMER,BALMER_ERR,Z_PP04_N2,Z_PP04_O3N2,AGNCLASS,SERSIC,R50_ARCSEC,R90_ARCSEC,R50_KPC,SIZE_REF,FLAG_ALMAQUEST,FLAG_EDGE,FLAG_SAMIALMA,FLAG_ALMAARCHIVE,FLAG_PACS,FLAG_SPIRE,FLAG_HELP,FLAG_NVSS,FLUX_20CM,FLUX_20CM_ERR,FLAG_WISE,FLAG_HI,HIREF,LOGMHI,ArrayConfig,SB_num,IAUname
int64,str23,str11,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,float32,float32,float32,float32,int16,int16,int16,int16,int16,int16,int16,int16,int16,float32,float32,int16,int16,int16,float32,str2,int64,str20
1,1-178894,MANGA,314.5881042480469,-0.35113999247550964,0.05285,10.623,0.02,-2.245,0.639,10.609994,-1.23214,10.623,-2.245,-0.081225544,-0.0008421099,1.5782075,0.3452157,9.213254,8.704277,2,4.8857927,2.320328,7.4042645,2.3893187,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,-9,-9,-99.0,C2,0,J205821.15-002104.10
2,1-179173,MANGA,314.5533752441406,-0.3243899941444397,0.05264,10.668,0.035,-0.816,1.01,10.667042,-1.2501199,10.668,-0.816,-0.036621608,0.18301708,2.2594492,0.5519009,9.297333,8.659715,3,3.9337363,2.764213,7.607959,2.8357954,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,-9,-9,-99.0,C2,0,J205812.81-001927.80
3,1-179071,MANGA,313.7587890625,-0.039969999343156815,0.05249,10.465,0.051,0.563,0.114,10.443735,0.9133401,10.465,0.563,-0.4191595,-0.28347042,4.874192,0.08326789,8.716915,8.686579,1,3.0689113,6.1513886,16.195345,6.293819,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,-9,-9,-99.0,C2,0,J205502.11-000223.89
4,1-178794,MANGA,310.8828430175781,0.963379979133606,0.05089,10.655,0.011,-0.989,0.211,10.63279,-1.31125,10.655,-0.989,0.16089667,nan,nan,nan,9.730572,nan,0,3.101069,2.1478245,6.2456813,2.134607,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,-9,-9,-99.0,C2,0,J204331.88+005748.17
5,1-54940,MANGA,144.53671264648438,1.4491599798202515,0.04578,10.46,0.019,-0.191,0.103,10.366038,-0.107449904,10.46,-0.191,-0.11807627,0.83848697,4.82871,0.38870245,9.147346,8.4239,4,0.54942983,5.0828276,9.722165,4.571927,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,2,2,10.072984,C2,1,J093808.81+012656.98
6,1-277,MANGA,146.51539611816406,0.16681000590324402,0.04892,9.935,0.05,-0.016,0.105,9.808366,0.104650095,9.935,-0.016,-0.44932115,-0.3503872,3.8629534,0.12038048,8.683231,8.69834,1,1.3003049,5.8137746,14.262073,5.5673018,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,1,2,10.07,C2,1,J094603.70+001000.52
7,1-55227,MANGA,146.57606506347656,2.8843400478363037,0.04746,10.353,0.05,0.424,0.226,10.229027,0.3536601,10.353,0.424,-0.4538082,-0.6710217,3.3505013,0.0692256,8.67835,8.799508,1,1.1034205,3.197625,6.8815346,2.9758244,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,2,2,9.977614,C2,1,J094618.26+025303.62
8,1-954,MANGA,149.20582580566406,1.1205099821090698,0.04705,10.46,0.014,-0.984,0.123,10.476883,-1.2375499,10.46,-0.984,0.029170804,0.20821443,1.8112577,0.52786016,9.430296,8.672706,3,6.0,3.053145,8.4320755,2.8181906,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,2,2,10.132503,C2,1,J095649.40+010713.84
9,1-383,MANGA,149.11256408691406,0.12973999977111816,0.04579,10.173,0.042,0.191,0.163,10.053984,0.12894009,10.173,0.191,-0.6453269,-0.030143593,4.4053283,0.14864746,8.498711,8.533141,1,1.5694531,4.350979,10.877237,3.914449,1,0,0,0,0,0,0,0,-9,-99.0,-99.0,1,1,2,10.05,C2,1,J095627.02+000747.06


In [17]:
list_columns(tab.colnames)

KGAS_ID               LOGSFR_SDSS           R50_ARCSEC            FLAG_NVSS             
ID                    LOGMSTAR_BEST         R90_ARCSEC            FLUX_20CM             
SURVEY                LOGSFR_BEST           R50_KPC               FLUX_20CM_ERR         
RA                    N2HA                  SIZE_REF              FLAG_WISE             
DEC                   O3HB                  FLAG_ALMAQUEST        FLAG_HI               
Z                     BALMER                FLAG_EDGE             HIREF                 
LOGMSTAR_GSWLC        BALMER_ERR            FLAG_SAMIALMA         LOGMHI                
LOGMSTAR_ERR_GSWLC    Z_PP04_N2             FLAG_ALMAARCHIVE      ArrayConfig           
LOGSFR_GSWLC          Z_PP04_O3N2           FLAG_PACS             SB_num                
LOGSFR_ERR_GSWLC      AGNCLASS              FLAG_SPIRE            IAUname               
LOGMSTAR_SDSS         SERSIC                FLAG_HELP                                   


In [19]:
mask_sami = (tab['SURVEY'] == 'SAMI')
mask_manga = (tab['SURVEY'] == 'MANGA')
tab_s_SAMI = tab[mask_sami]
tab_s_MaNGA = tab[mask_manga]

In [21]:
print(f'# MaNGA {len(tab_s_MaNGA)}, # SAMI {len(tab_s_SAMI)}')

# MaNGA 283, # SAMI 169


In [22]:
sami_names = []
for tab_now in tab_s_SAMI:
    idx = tab_now['ID']
    sami_names.append(f'SAMI{idx}')
sami_names = np.array(sami_names)

def write_numpy_array_to_ascii(array: np.ndarray, file_path: str):
    """
    Writes a NumPy string array to an ASCII file line by line.
    
    :param array: NumPy array of strings to write
    :param file_path: Destination ASCII file path
    """
    if not isinstance(array, np.ndarray) or array.dtype.type is not np.str_:
        raise ValueError("Input must be a NumPy array of strings.")
    
    with open(file_path, 'w') as file:
        for line in array:
            file.write(line + '\n')

# Call the function
write_numpy_array_to_ascii(sami_names, 'tables/sami_kilogas.txt')

#print(sami_names)

In [23]:
list(tab_s_SAMI['ID'].astype(int))

[205168,
 210070,
 205166,
 601018,
 575097,
 197866,
 204832,
 218713,
 623620,
 302810,
 594049,
 93020,
 92944,
 593680,
 93604,
 107214,
 93062,
 93023,
 78827,
 509075,
 240432,
 493028,
 63302,
 63777,
 485882,
 485919,
 505341,
 485924,
 619095,
 346046,
 323575,
 278846,
 278773,
 550330,
 203729,
 371889,
 202531,
 549313,
 214860,
 371976,
 301381,
 346752,
 346793,
 514022,
 618144,
 463641,
 460767,
 319049,
 319018,
 91627,
 618152,
 62244,
 485688,
 278840,
 323577,
 371170,
 376033,
 301191,
 323855,
 550328,
 387715,
 623144,
 418693,
 372384,
 324323,
 601102,
 386892,
 376990,
 601013,
 600929,
 536625,
 622084,
 208520,
 209279,
 301201,
 204096,
 543797,
 15847,
 319397,
 227961,
 463367,
 618220,
 319452,
 238406,
 238282,
 238276,
 570197,
 594946,
 79850,
 505979,
 570119,
 570174,
 594906,
 487175,
 79693,
 79810,
 486834,
 595088,
 594986,
 509852,
 65384,
 49784,
 513236,
 573586,
 202636,
 549182,
 381229,
 599310,
 202398,
 202480,
 202673,
 300406,
 300411,

In [24]:
tab_s_SAMI['ID','Z'][tab_s_SAMI['ID'] == '298666']

ID,Z
str23,float32
298666,0.02795


In [60]:
tab_s_SAMI['ID','Z'].write('tables/s_SAMI.csv', format='csv', overwrite=True)

In [27]:
print(len(tab_MaNGA['name','ID']))
name_un = np.unique(tab_MaNGA['name'])
ID_un = np.unique(tab_MaNGA['ID'])
print(len(name_un),len(ID_un))



10220
10220 10081


In [25]:
#
# MaNGA files
#
tab_s_MaNGA_dp = tab_MaNGA['name','ID']

tab_s_MaNGA_j = Table_join(tab_s_MaNGA,tab_s_MaNGA_dp,keys='ID',join_type='inner')
print(len(tab_s_MaNGA_dp))
print(len(tab_s_MaNGA))
print(len(tab_s_MaNGA_j))



10220
283
285


In [69]:
tab_s_MaNGA_j['name','ID','Z'].write('tables/s_MaNGA.csv', format='csv', overwrite=True)

In [ ]:
#
#hdu=fits.open('tables/KILOGAS_global_catalog.fits')
# tab=Table(hdu[1].data)     

#

In [71]:
import astropy
print(f"Astropy version: {astropy.__version__}")
print("Supported formats:", [f for f in astropy.io.registry.get_formats()['Format'] if 'ecsv' in f])


Astropy version: 5.3.4
Supported formats: ['ascii.ecsv']
